# Codice per rendere visibile lo svolgimento del progetto nell'evenualità che il link di databricks non funzionasse.

In [ ]:
!wget https://proai-datasets.s3.eu-west-3.amazonaws.com/bitcoin_tweets.csv

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType
from langdetect import detect
from textblob import TextBlob
from pyspark.sql.functions import sum
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql.functions import avg

plt.style.use("seaborn-v0_8-darkgrid")
plt.rcParams["figure.figsize"] = [10, 6]


def detect_language(text):
    """
    Dato un testo restituisce la lingua d'appartenenza
    text = str
    """
    try:
        return detect(text)
    except:
        return None


def sentiment_analysis(text):
    """
    Dato un testo restituisce il sentiment, compreso tra -1 e 1
    text = str
    """
    try:
        sentiment = TextBlob(text).sentiment
        return float(sentiment.polarity)
    except:
        return None

In [ ]:
import pandas as pd
dataset = pd.read_csv("/databricks/driver/bitcoin_tweets.csv", usecols=[4,5,6,7,8], delimiter=",")

spark_df = spark.createDataFrame(dataset)
# spark_df.write.saveAsTable("bitcoin_tweets")

In [ ]:
display(spark_df)

Considerando la grande dimensionalità del dataset e i tempi computazionali necessari, viene selezionato un campione composto da delle osservazioni estratte casualmente per poter comporre una ridotta percentuale del medesimo dataset. Tutte le manipolazioni del dato rimangono comunque valide per manipolare il dataset originale.

In [ ]:
spark_df.count()

In [ ]:
dataset_sample = spark_df.sample(False, 0.005)
dataset_sample.count()

Per l'analisi del sentimento sul tema del BTC verrà usato un modello di machine learning pre-addestrato (TextBlob), lo stesso è ottimizzato per la lingua inglese. Per questo caso è utile selezionare i record che comprendono il campo "text" (il tweet) in lingua inglese. Lo scopo viene raggiunto con l'utilizzo di UDF per estendere le funzionalità base e accedere a delle funzioni personalizzate. In questo caso dopo aver registrato l'UDF con la funzione che utilizza la libreria langdetect per il rilevamento della lingua, viene creato un campo che viene valorizzato con la lingua per ogni testo, quindi per ogni record.

In [ ]:

language_udf = udf(detect_language, StringType())
dataset_sample_language = dataset_sample.withColumn("language", language_udf(dataset_sample["text"]))

dataset_sample_language.show()

Selezione dei record in lingua inglese con l'utilizzo di una mascehra booleana.

In [ ]:
dataset_en = dataset_sample_language[dataset_sample_language["language"] == "en"]

Registrazione dell'UDF con la funzione che prendendo come argomento il testo del campo "text" restituirà il sentiment compreso tra i valori -1 e 1, dove -1 equivale per un sentiment totalmente negativo, 0 un sentiment neutrale e 1 per un sentiment totalmente positivo. La funzione che permette ciò utilizza il modello pre-addestrato TextBlob. Infine viene creato un campo con tale valore.

In [ ]:
sentiment_udf = udf(sentiment_analysis, DoubleType())
sentiment_sample_language = dataset_en.withColumn("sentiment", sentiment_udf(dataset_en["text"]))

Quesiti:

i tweet negativi hanno avuto più likes rispetto a quelli positivi?

i tweet negativi hanno avuto più interazioni (risposte) rispetto a quelli positivi?

Utilizzo di una mascehra booleana per filtare solamente i valori positivi e negativi del dataset. Vengno impostati dei valori di soglia per i tweet positivi e negativi, rispettivamente 0.25 e -0.25.

In [ ]:
sentiment_positive = sentiment_sample_language[sentiment_sample_language["sentiment"] >= 0.25]
sentiment_negative = sentiment_sample_language[sentiment_sample_language["sentiment"] <= -0.25]

Calcolo della somma dei valori nel campo "likes" e del campo "replies" che sono l'oggetto del quesito.

In [ ]:
positive_tweet = sentiment_positive.agg(sum("likes").alias("total_likes")).collect()[0]["total_likes"]
negative_tweet = sentiment_negative.agg(sum("likes").alias("total_likes")).collect()[0]["total_likes"]

positive_replies = sentiment_positive.agg(sum("replies").alias("total_replies")).collect()[0]["total_replies"]
negative_replies = sentiment_negative.agg(sum("replies").alias("total_replies")).collect()[0]["total_replies"]

Visualizzazione dei risultati in un grafico a barre raggruppate, divisi per risposte e likes dei tweets sono visualizzate le numerosità per ogni classe, sia positiva che negativa.

In [ ]:
categories = ("Risposte", "Likes")
sentiment = {
    "Tweets positivi": (positive_replies, positive_tweet),
    "Tweets negativi": (negative_replies, negative_tweet),
}

x = np.arange(len(categories))
width = 0.30
multiplier = 0

fig, ax = plt.subplots(layout="constrained")

for attribute, measurement in sentiment.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1


ax.set_ylabel("Numerosità")
ax.set_title("Apprezzamento BTC basato sui Tweet")
ax.set_xticks(x + width /2, categories)
ax.legend(loc="upper left", ncols=3)
ax.set_ylim(0, max(positive_replies, positive_tweet, negative_replies, negative_tweet)+1000)

plt.show()

Quesito:

il tuo compito è quello di eseguire un'analisi del sentiment e creare un grafico che mostri come questo è variato giorno per giorno.

La rappresentazione sul grafico del periodo totale disponibile dai dati risutla essere confuso e poco comprensibile per via delle troppe informazioni nel poco spazio rappresentabile, quindi viene selezionato arbitrariamente un periodo che consente una rappresentazione comprensibile, in questo caso ho scelto il mese di Gennaio 2018.

Il dataset viene inizialmente raggruppato per il campo "timestamp" contenente le date e viene calcolata la media dei valori del campo "sentiment" cosi raggruppati per ottenere la media giornaliera.

Dopo aver converito i valori dell campo "timestamp" nel formato data viene utilizzata una maschera booleana per selezionare solamente i record nella fascia d'interesse. Infine, il dataset viene ordinato.

In [ ]:
dataset_groupby = sentiment_sample_language.groupBy("timestamp").agg(
    avg("sentiment").alias("avg_sentiment")
)

dataset_pandas = dataset_groupby.toPandas()
dataset_pandas["date"] = pd.to_datetime(dataset_pandas["timestamp"], format="%Y-%m-%d")

dataset_pandas_2018 = dataset_pandas[(dataset_pandas["date"] >= "2018-01-01") & (dataset_pandas["date"] <= "2018-01-31")]

dataset_pandas_2018.sort_values("date", inplace=True)
dataset_pandas_2018.set_index("date", inplace=True)

Visualizzazione di un grafico a linea con la media del sentiment che mostra l'oscillazione dell'apprezzamento sul tema del BTC da parte della comunità di Twitter nel primo mese del 2018.

In [ ]:
plt.plot(dataset_pandas_2018.index, dataset_pandas_2018["avg_sentiment"], marker="o", label="Media Sentiment")
plt.ylabel("Sentiment")
plt.title("Sentiment BTC di Gennaio 2018")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()